# Drought Propagation from Meteorological to Hydrological using Cross-Correlation

The notebook explain on how to analyzing the propagation of **meteorological drought** (Standardized Precipitation Index - SPI) to **hydrological drought** (Standardized Streamflow Index - SSI) using **Cross-Correlation Analysis (CCA)** at the pixel level. It required two input netCDF files, SPI - generated from `1_Steps_to_Generate_SPI_Using_CHIRPS_Data.ipynb` and SSI - generated from `2_Steps_to_Generate_SSI_Using_GloFAS-ERA5_Data.ipynb`.

As background, we have SPI and SSI data which has same temporal resolution (monthly) and cover the same period (1981-2022), also has same various time scale: `1`, `2`, `3`, `6`, `9`, `12`, `18`, `24`, `36`, `48`, `60` and `72-month`. 


The analysis **ideally** will take some process.

1. **Choice of Time Scale:**

    Let's start with the 3, 6, 9 and 12-month timescale. 

2. **Handling Seasonality:**

    While splitting the dataset by season can be complex due to varying wet and dry seasons across regions, we might consider a simpler approach: analyze the entire dataset first, then conduct a separate analysis for regions with different climatic patterns. Another method is to detrend the data to remove seasonality, but this might also remove some meaningful variability related to the drought dynamics. **For simplicity, we will not do this part.**

3. **Noise Filtering:**

    Employing noise filtering techniques like Singular Spectrum Analysis (SSA) or a moving average can indeed help in isolating the underlying trends and patterns in our data. This step is crucial for enhancing the signal-to-noise ratio in our datasets.

4. **Cross-Correlation Analysis:**

    Perform the cross-correlation for each pixel across the entire time series. This will involve computing the correlation for different lags (e.g., 1 month, 2 months, up to a reasonable maximum that makes sense for our study).

5. **Output and Visualization**

    **Lag Map:** This will be particularly insightful, showing the spatial distribution of the lag between meteorological and hydrological droughts.

    **Strength Map:** This map will be useful in understanding where the correlations are strongest, indicating regions where meteorological conditions are good predictors of hydrological conditions.


We will utilise some [Python](https://www.python.org/) library like `xarray`, `pandas`, `numpy`, `statsmodel`, `matplotlib`, `cartopy`, `tqdm` and other necessary tools. Assume we are working inside a dedicated Python/Conda environment, so it will not break other configuration.

## 0. Working Directory

For this exercise, I am working on these folder `/Temp/drought/prop/` (applied to Mac/Linux machine) or `Z:/Temp/drought/prop/` (applied to Windows machine) directory. I have some folder inside this directory:

1. `01_event` # Result from converting the SPI or SSI into drought event.
2. `02_magnitude` # Result from converting the SPI or SSI and drought event into drought magnitude (cumulative SPI or SPI value during drought event).
3. `03_filtered` # Result from noise filtering for drought magnitude dataset.
4. `04_correlation` # Result from cross-correlation using filtered dataset for each lag.
5. `05_frequency` # Result from maximum/mode correlation and on which lag the maximum/mode correlation is happen.
6. `images` # In this folder I put some screenshot file as illustration, used in this notebook.

Feel free to use your own preferences for this setting/folder arrangements.

This step-by-step guide was tested using Windows 11 with WSL2 - Ubuntu 22 enabled running on Thinkpad T480 2019, i7-8650U 1.9GHz, 64 GB 2400 MHz DDR4.

## 1. Preprocessing

The drought characteristics originally following the method proposed by Yevjevich in [1967](https://www.engr.colostate.edu/ce/facultystaff/yevjevich/papers/HydrologyPapers_n23_1967.pdf) and has been employed to recognize the feature of droughts. The paper from Le, et al in [2019](https://www.researchgate.net/publication/333171255_Space-time_variability_of_drought_over_Vietnam) provide better explanation about it: duration, severity, intensity, and interarrival.

![Drought](./prop/images/drought-runtheory.png)

The drought condition is set when the SPI or SSI value negative, or less than `-1.2`. Focusing on drought conditions could be a more relevant approach for our analysis compare to using all SPI and SSI data, as it has dry and wet condition. By concentrating on these periods, we can potentially gain more insight into the correlation between meteorological and hydrological droughts.

**Masking for Drought Event:**

Create masks for both SPI and SSI datasets where only values less than -1.2 are retained. This step isolates the drought conditions.


In [1]:
import xarray as xr
import os
import numpy as np

# Function to save dataset with automatic overwrite
def save_dataset(ds, filename):
    ds.to_netcdf(filename)
    print(f"File saved as '{filename}'.")

# Define folders and subset file path
spi_folder = './met/05_spi/'
ssi_folder = './hyd/08_ssi/'
event_folder = './prop/01_event/'
subset_file = './subset/idn_subset_chirps.nc'

# Ensure the event folder exists
os.makedirs(event_folder, exist_ok=True)

# Load the land mask from the subset file
land_mask = xr.open_dataset(subset_file)['land']

# Define time scales
time_scales = [3, 6, 9, 12]

# Process datasets for both SPI and SSI for each time scale
for time_scale in time_scales:
    print(f"Processing time scale: {time_scale} months")

    # SPI file processing
    spi_file = spi_folder + f'idn_cli_spi_gamma_{time_scale}_month.nc'
    print(f"  Loading SPI data from '{spi_file}'")
    ds_spi = xr.open_dataset(spi_file)
    print("  Calculating drought events for SPI...")
    ds_spi['drought_event'] = xr.where(ds_spi[f'spi_gamma_{time_scale}_month'] < -1.2, 1, 0)
    ds_spi['drought_event'] = ds_spi['drought_event'].where(land_mask == 1, other=np.nan)
    drought_event_spi_file = event_folder + f'idn_cli_drought_event_spi_{time_scale}_month.nc'
    save_dataset(ds_spi, drought_event_spi_file)

    # SSI file processing
    ssi_file = ssi_folder + f'idn_cli_ssi_gamma_{time_scale}_month.nc'
    print(f"  Loading SSI data from '{ssi_file}'")
    ds_ssi = xr.open_dataset(ssi_file)
    print("  Calculating drought events for SSI...")
    ds_ssi['drought_event'] = xr.where(ds_ssi[f'ssi_gamma_{time_scale}_month'] < -1.2, 1, 0)
    ds_ssi['drought_event'] = ds_ssi['drought_event'].where(land_mask == 1, other=np.nan)
    drought_event_ssi_file = event_folder + f'idn_cli_drought_event_ssi_{time_scale}_month.nc'
    save_dataset(ds_ssi, drought_event_ssi_file)

    print(f"Drought event calculation completed for SPI and SSI, time scale {time_scale} months")

print("Completed!")


Processing time scale: 3 months
  Loading SPI data from './met/05_spi/idn_cli_spi_gamma_3_month.nc'
  Calculating drought events for SPI...
File saved as './prop/01_event/idn_cli_drought_event_spi_3_month.nc'.
  Loading SSI data from './hyd/08_ssi/idn_cli_ssi_gamma_3_month.nc'
  Calculating drought events for SSI...
File saved as './prop/01_event/idn_cli_drought_event_ssi_3_month.nc'.
Drought event calculation completed for SPI and SSI, time scale 3 months
Processing time scale: 6 months
  Loading SPI data from './met/05_spi/idn_cli_spi_gamma_6_month.nc'
  Calculating drought events for SPI...
File saved as './prop/01_event/idn_cli_drought_event_spi_6_month.nc'.
  Loading SSI data from './hyd/08_ssi/idn_cli_ssi_gamma_6_month.nc'
  Calculating drought events for SSI...
File saved as './prop/01_event/idn_cli_drought_event_ssi_6_month.nc'.
Drought event calculation completed for SPI and SSI, time scale 6 months
Processing time scale: 9 months
  Loading SPI data from './met/05_spi/idn_cli_

**Calculate Drought Magnitude:**

Compute the absolute cumulative values during drought events for both datasets. This gives a measure of drought magnitude, which may be more meaningful for correlation analysis than using raw SPI/SSI values.

This script reads the drought event data and the absolute values, calculates the cumulative magnitude during drought periods, and writes the results to new NetCDF files.

In [2]:
import xarray as xr
import numpy as np
from tqdm import tqdm
import os

# Define folders and subset file path
spi_folder = './met/05_spi/'
ssi_folder = './hyd/08_ssi/'
event_folder = './prop/01_event/'
magnitude_folder = './prop/02_magnitude/'
subset_file = './subset/idn_subset_chirps.nc'

# Ensure the magnitude folder exists
os.makedirs(magnitude_folder, exist_ok=True)

# Load the land mask from the subset file
print("Loading land mask...")
land_mask = xr.open_dataset(subset_file)['land']

# Define time scales
time_scales = [3, 6, 9, 12]

# Calculate drought magnitude for each time scale
for time_scale in time_scales:
    print(f"\nProcessing time scale: {time_scale} months")
    
    # Process SPI and SSI files
    for data_type, folder in [('spi', spi_folder), ('ssi', ssi_folder)]:
        print(f"  Processing {data_type.upper()} data...")

        # Open the drought event data
        event_file = event_folder + f'idn_cli_drought_event_{data_type}_{time_scale}_month.nc'
        print(f"    Opening drought event file: {event_file}")
        ds_drought = xr.open_dataset(event_file)

        # Open the corresponding SPI/SSI data
        data_file = folder + f'idn_cli_{data_type}_gamma_{time_scale}_month.nc'
        print(f"    Opening {data_type.upper()} data file: {data_file}")
        ds_data = xr.open_dataset(data_file)

        # Transpose datasets to respect original dimensions
        ds_drought = ds_drought.transpose('time', 'lat', 'lon')
        ds_data = ds_data.transpose('time', 'lat', 'lon')

        # Apply land mask to drought event data
        print("    Applying land mask to drought event data...")
        ds_drought['drought_event'] = ds_drought['drought_event'].where(land_mask == 1, other=np.nan)

        # Preallocate an empty array for drought_magnitude
        drought_magnitude_arr = np.empty_like(ds_drought['drought_event'].values, dtype=float)
        drought_magnitude_arr[:] = np.nan

        # Calculate drought magnitude
        print("    Calculating drought magnitude...")
        for i in tqdm(range(1, len(ds_drought.time)), desc="      Processing", unit=" timestep"):
            drought_event = ds_drought['drought_event'].isel(time=i).values
            data_value = ds_data[f'{data_type}_gamma_{time_scale}_month'].isel(time=i).values
            drought_magnitude_arr[i] = np.where(drought_event == 1, abs(data_value) + drought_magnitude_arr[i-1], 0)

        # Create a new DataArray for drought magnitude
        print("    Create a new DataArray for drought magnitude...")
        drought_magnitude_da = xr.DataArray(drought_magnitude_arr, 
                                            coords={'time': ds_drought.time, 'lat': ds_drought.lat, 'lon': ds_drought.lon}, 
                                            dims=['time', 'lat', 'lon'],
                                            name=f'drought_magnitude_{data_type}')

        # Apply land mask to the calculated drought magnitude
        drought_magnitude_da = drought_magnitude_da.where(land_mask == 1, other=np.nan)

        # Set attributes according to CF Conventions
        drought_magnitude_da.attrs['standard_name'] = f'drought_magnitude_{data_type}'
        drought_magnitude_da.attrs['long_name'] = f'Drought Magnitude for {data_type.upper()}, {time_scale}-month timescale'
        drought_magnitude_da.attrs['units'] = '1'  # Assuming unitless

        # Create a new dataset
        ds_magnitude = xr.Dataset({f'drought_magnitude_{data_type}': drought_magnitude_da})

        # Save the new drought magnitude variable to a new netCDF file
        output_file = magnitude_folder + f'idn_cli_drought_magnitude_{data_type}_{time_scale}_month.nc'
        print(f"    Saving drought magnitude file: {output_file}")
        ds_magnitude.to_netcdf(output_file)

        print(f"  Drought magnitude calculations completed for {data_type.upper()}, time scale {time_scale} months")

print("\nAll processes completed!")


Loading land mask...

Processing time scale: 3 months
  Processing SPI data...
    Opening drought event file: ./prop/01_event/idn_cli_drought_event_spi_3_month.nc
    Opening SPI data file: ./met/05_spi/idn_cli_spi_gamma_3_month.nc
    Applying land mask to drought event data...
    Calculating drought magnitude...


      Processing: 100%|████████████████████████████████████████████████████████████████| 503/503 [49:36<00:00,  5.92s/ timestep]


    Create a new DataArray for drought magnitude...
    Saving drought magnitude file: ./prop/02_magnitude/idn_cli_drought_magnitude_spi_3_month.nc
  Drought magnitude calculations completed for SPI, time scale 3 months
  Processing SSI data...
    Opening drought event file: ./prop/01_event/idn_cli_drought_event_ssi_3_month.nc
    Opening SSI data file: ./hyd/08_ssi/idn_cli_ssi_gamma_3_month.nc
    Applying land mask to drought event data...
    Calculating drought magnitude...


      Processing: 100%|████████████████████████████████████████████████████████████████| 503/503 [46:09<00:00,  5.51s/ timestep]


    Create a new DataArray for drought magnitude...
    Saving drought magnitude file: ./prop/02_magnitude/idn_cli_drought_magnitude_ssi_3_month.nc
  Drought magnitude calculations completed for SSI, time scale 3 months

Processing time scale: 6 months
  Processing SPI data...
    Opening drought event file: ./prop/01_event/idn_cli_drought_event_spi_6_month.nc
    Opening SPI data file: ./met/05_spi/idn_cli_spi_gamma_6_month.nc
    Applying land mask to drought event data...
    Calculating drought magnitude...


...

    Create a new DataArray for drought magnitude...
    Saving drought magnitude file: ./prop/02_magnitude/idn_cli_drought_magnitude_ssi_9_month.nc
  Drought magnitude calculations completed for SSI, time scale 9 months

Processing time scale: 12 months
  Processing SPI data...
    Opening drought event file: ./prop/01_event/idn_cli_drought_event_spi_12_month.nc
    Opening SPI data file: ./met/05_spi/idn_cli_spi_gamma_12_month.nc
    Applying land mask to drought event data...
    Calculating drought magnitude...


      Processing: 100%|████████████████████████████████████████████████████████████████| 503/503 [48:52<00:00,  5.83s/ timestep]


    Create a new DataArray for drought magnitude...
    Saving drought magnitude file: ./prop/02_magnitude/idn_cli_drought_magnitude_spi_12_month.nc
  Drought magnitude calculations completed for SPI, time scale 12 months
  Processing SSI data...
    Opening drought event file: ./prop/01_event/idn_cli_drought_event_ssi_12_month.nc
    Opening SSI data file: ./hyd/08_ssi/idn_cli_ssi_gamma_12_month.nc
    Applying land mask to drought event data...
    Calculating drought magnitude...


      Processing: 100%|████████████████████████████████████████████████████████████████| 503/503 [48:17<00:00,  5.76s/ timestep]


    Create a new DataArray for drought magnitude...
    Saving drought magnitude file: ./prop/02_magnitude/idn_cli_drought_magnitude_ssi_12_month.nc
  Drought magnitude calculations completed for SSI, time scale 12 months

All processes completed!


## 2. Noise Filtering with Singular Spectrum Analysis

This part of our analysis focuses on applying Singular Spectrum Analysis (SSA) for noise filtering and trend extraction in drought magnitude data in SPI and SSI datasets. In drought propagation analysis, noise filtering with SSA is a critical step for data preparation. SSA effectively separates the underlying signal from the noise in climate datasets, such as Standardized Precipitation Index (SPI) and Standardized Streamflow Index (SSI). This process is crucial for enhancing the clarity and accuracy of the data, which in turn facilitates a more precise understanding of drought patterns and their progression. 

By filtering out the noise, SSA aids in identifying genuine drought trends and magnitudes, providing a reliable basis for subsequent analysis of drought propagation and its potential impacts. This enhanced data quality is instrumental in developing effective drought management strategies and improving predictive models. 

Key steps and features of this script include:

* **SSA Implementation:** Utilizes the `seasonal_decompose` function from the `statsmodels` library for SSA, effectively decomposing time series data into trend, seasonal, and residual components.
* **Data Preparation and Processing:**
    * Converts xarray DataArrays to pandas DataFrames for SSA application.
    * Handles missing values in SSI data through forward filling, ensuring data continuity.
* **Trend Extraction:** Focuses on extracting the trend component from both SPI and SSI data, considered as the crucial signal of interest.
* **Data Transformation:** Transforms the extracted trend components back into xarray DataArrays for further analysis.
* **Metadata Assignment:** Assigns appropriate metadata to the trend components, enhancing data understanding and context.
* **NetCDF File Output:** Saves the filtered datasets as NetCDF files, adhering to the Climate and Forecast (CF) Conventions, making them ready for subsequent analysis or visualization.


In [3]:
import xarray as xr
import pandas as pd
from tqdm.auto import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
import os

# Register tqdm with pandas to enable progress_apply
tqdm.pandas()

# Function to apply SSA
def apply_ssa(data, freq):
    result = seasonal_decompose(data.dropna(), model='additive', period=freq)
    return result.trend

# Function to save dataset with CF Conventions
def save_dataset(dataset, filename, data_type, time_scale):
    if os.path.exists(filename):
        print(f"    Overwriting existing file: {filename}")
        os.remove(filename)

    # Set attributes according to CF Conventions
    dataset[f'drought_magnitude_{data_type}_filtered'].attrs = {
        'standard_name': f'drought_magnitude_{data_type}_filtered',
        'long_name': f'Drought Magnitude of {data_type.capitalize()}, {time_scale}-month (Filtered)',
        'units': '1',  # SPI and SSI are unitless
        'comment': 'Filtered using Singular Spectrum Analysis'
    }
    dataset.to_netcdf(filename)
    print(f"    File saved as '{filename}'.")

# Define folders and subset file path
magnitude_folder = './prop/02_magnitude/'
subset_file = './subset/idn_subset_chirps.nc'
filtered_folder = './prop/03_filtered/'

# Ensure the filtered folder exists
os.makedirs(filtered_folder, exist_ok=True)

# Load the land mask from the subset file
land_mask = xr.open_dataset(subset_file)['land']

# Define time scales and data types
time_scales = [3, 6, 9, 12]
data_types = ['spi', 'ssi']

for time_scale in time_scales:
    for data_type in data_types:
        print(f"\nProcessing {data_type.upper()} data for time scale: {time_scale} months")

        # Load the dataset
        print("    Load the dataset...")
        file_path = magnitude_folder + f'idn_cli_drought_magnitude_{data_type}_{time_scale}_month.nc'
        ds = xr.open_dataset(file_path)
        ds[f'drought_magnitude_{data_type}'] = ds[f'drought_magnitude_{data_type}'].where(land_mask == 1, other=np.nan)

        # Select data starting from the appropriate month
        print("    Select the time period...")
        start_year = '1981'
        start_date = f"{start_year}-{str(time_scale).zfill(2)}-01"
        ds_selected = ds.sel(time=slice(start_date, '2022-12-31'))

        # Convert to pandas DataFrame
        print("    Convert to pandas DataFrame...")
        df = ds_selected.to_dataframe().unstack('time')[f'drought_magnitude_{data_type}']
        df_filled = df.fillna(method='ffill').fillna(method='bfill')

        # Apply SSA with tqdm progress bar
        print("    Applying SSA for noise filtering...")
        df_filtered = df_filled.progress_apply(lambda x: apply_ssa(x, freq=12), axis=0)

        # Convert back to xarray DataArray
        print("    Convert back to xarray DataArray...")
        df_filtered = df_filtered.stack().reset_index()
        df_filtered = df_filtered.rename(columns={'level_0': 'lat', 'level_1': 'lon', 0: 'filtered_data'})
        filtered_da = df_filtered.set_index(['time', 'lat', 'lon']).to_xarray()['filtered_data']

        # Realign the land mask with the filtered data
        aligned_land_mask = land_mask.reindex_like(filtered_da, method='nearest')

        # Apply the aligned land mask to the filtered data
        filtered_da_masked = filtered_da.where(aligned_land_mask == 1, other=np.nan)

        # Create xarray Dataset
        print("    Create xarray Dataset...")
        ds_filtered = xr.Dataset({f'drought_magnitude_{data_type}_filtered': filtered_da_masked})

        # Save the filtered dataset as NetCDF file with attributes
        output_file = filtered_folder + f'idn_cli_drought_magnitude_{data_type}_{time_scale}_month_filtered.nc'
        print(f"    Saving filtered dataset: {output_file}")
        save_dataset(ds_filtered, output_file, data_type, time_scale)

print("\nAll processing completed.")



Processing SPI data for time scale: 3 months
    Load the dataset...
    Select the time period...
    Convert to pandas DataFrame...
    Applying SSA for noise filtering...


  0%|          | 0/502 [00:00<?, ?it/s]

    Convert back to xarray DataArray...
    Create xarray Dataset...
    Saving filtered dataset: ./prop/03_filtered/idn_cli_drought_magnitude_spi_3_month_filtered.nc
    Overwriting existing file: ./prop/03_filtered/idn_cli_drought_magnitude_spi_3_month_filtered.nc
    File saved as './prop/03_filtered/idn_cli_drought_magnitude_spi_3_month_filtered.nc'.

Processing SSI data for time scale: 3 months
    Load the dataset...
    Select the time period...
    Convert to pandas DataFrame...
    Applying SSA for noise filtering...


  0%|          | 0/502 [00:00<?, ?it/s]

    Convert back to xarray DataArray...
    Create xarray Dataset...
    Saving filtered dataset: ./prop/03_filtered/idn_cli_drought_magnitude_ssi_3_month_filtered.nc
    Overwriting existing file: ./prop/03_filtered/idn_cli_drought_magnitude_ssi_3_month_filtered.nc
    File saved as './prop/03_filtered/idn_cli_drought_magnitude_ssi_3_month_filtered.nc'.

Processing SPI data for time scale: 6 months
    Load the dataset...
    Select the time period...
    Convert to pandas DataFrame...
    Applying SSA for noise filtering...


...

    Convert back to xarray DataArray...
    Create xarray Dataset...
    Saving filtered dataset: ./prop/03_filtered/idn_cli_drought_magnitude_ssi_9_month_filtered.nc
    File saved as './prop/03_filtered/idn_cli_drought_magnitude_ssi_9_month_filtered.nc'.

Processing SPI data for time scale: 12 months
    Load the dataset...
    Select the time period...
    Convert to pandas DataFrame...
    Applying SSA for noise filtering...


  0%|          | 0/493 [00:00<?, ?it/s]

    Convert back to xarray DataArray...
    Create xarray Dataset...
    Saving filtered dataset: ./prop/03_filtered/idn_cli_drought_magnitude_spi_12_month_filtered.nc
    File saved as './prop/03_filtered/idn_cli_drought_magnitude_spi_12_month_filtered.nc'.

Processing SSI data for time scale: 12 months
    Load the dataset...
    Select the time period...
    Convert to pandas DataFrame...
    Applying SSA for noise filtering...


  0%|          | 0/493 [00:00<?, ?it/s]

    Convert back to xarray DataArray...
    Create xarray Dataset...
    Saving filtered dataset: ./prop/03_filtered/idn_cli_drought_magnitude_ssi_12_month_filtered.nc
    File saved as './prop/03_filtered/idn_cli_drought_magnitude_ssi_12_month_filtered.nc'.

All processing completed.


## 3. Cross-Correlation Analysis

Cross-Correlation Analysis, especially when applied to data refined through Singular Spectrum Analysis (SSA) noise filtering, is pivotal in understanding drought propagation. This technique examines the relationship between different drought indicators across various time scales. 

By utilizing data filtered through SSA, which isolates the core signal from noise, Cross-Correlation Analysis can more accurately determine the time lag and intensity with which meteorological droughts (indicated by SPI) translate into hydrological droughts (indicated by SSI). This approach is essential for predicting the onset and progression of drought conditions, enabling timely decision-making and effective resource management to mitigate the adverse impacts of droughts.

Key steps and features of this script include:

* **Loading Data:** Imports meteorological and hydrological drought indices from NetCDF files.
* **Land Mask Application:** Utilizes a land mask to focus analysis on terrestrial areas, excluding ocean or sea regions.
* **Cross-Correlation Calculation:**
    * Computes the correlation for various time lags (up to 12 months).
    * Analyzes each grid cell separately, ensuring detailed spatial analysis.
* **Result Processing:** Identifies the time lag with the maximum correlation at each location, providing insights into drought propagation.
* **Data Array Creation:** Constructs xarray DataArrays for both lag time and correlation strength, including appropriate metadata.
* **NetCDF Output:** Saves the processed data as a NetCDF file, suitable for further analysis or visualization.


In [12]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Define folders
filtered_folder = './prop/03_filtered/'  # Filtered drought magnitude
correlation_base_folder = './prop/04_correlation/'
subset_file = './subset/idn_subset_chirps.nc'

# Load the land mask from the subset file
land_mask = xr.open_dataset(subset_file)['land']

# Define time scales and lags
time_scales = [3, 6, 9, 12]
max_lag = 12

# Function to calculate cross-correlation
def calculate_cross_correlation(magnitude_spi_data, magnitude_ssi_data, lag):
    corr_values = np.full((magnitude_spi_data.shape[1], magnitude_spi_data.shape[2]), np.nan)
    for lat in range(magnitude_spi_data.shape[1]):
        for lon in range(magnitude_spi_data.shape[2]):
            if land_mask[lat, lon]:  # Only calculate for land points
                magnitude_spi_series = magnitude_spi_data[:-lag or None, lat, lon]
                magnitude_ssi_series = magnitude_ssi_data[lag:, lat, lon]
                valid_idx = ~np.isnan(magnitude_spi_series) & ~np.isnan(magnitude_ssi_series)
                if valid_idx.any():
                    corr = np.corrcoef(magnitude_spi_series[valid_idx], magnitude_ssi_series[valid_idx])[0, 1]
                    corr_values[lat, lon] = corr
    return corr_values

# Function to align datasets to the larger time scale
def align_to_larger_time_scale(ds1, ds2, time_scale1, time_scale2):
    larger_time_scale = max(time_scale1, time_scale2)
    start_year = '1981'
    start_date = f"{start_year}-{str(larger_time_scale).zfill(2)}-01"
    end_date = '2022-12-31'
    ds1_aligned = ds1.sel(time=slice(start_date, end_date))
    ds2_aligned = ds2.sel(time=slice(start_date, end_date))
    return ds1_aligned, ds2_aligned

# Process each time scale combination
for magnitude_spi_time_scale in time_scales:
    for magnitude_ssi_time_scale in time_scales:
        time_scale_folder = f"spi{magnitude_spi_time_scale:02d}_ssi{magnitude_ssi_time_scale:02d}"
        correlation_folder = os.path.join(correlation_base_folder, time_scale_folder)
        os.makedirs(correlation_folder, exist_ok=True)

        print(f"Processing SPI (Time Scale: {magnitude_spi_time_scale}) and SSI (Time Scale: {magnitude_ssi_time_scale})")

        # Load drought magnitude datasets
        magnitude_spi_file = filtered_folder + f'idn_cli_drought_magnitude_spi_{magnitude_spi_time_scale}_month_filtered.nc'
        magnitude_ssi_file = filtered_folder + f'idn_cli_drought_magnitude_ssi_{magnitude_ssi_time_scale}_month_filtered.nc'
        magnitude_spi_ds = xr.open_dataset(magnitude_spi_file)
        magnitude_ssi_ds = xr.open_dataset(magnitude_ssi_file)

        # Align datasets to the larger time scale
        magnitude_spi_data, magnitude_ssi_data = align_to_larger_time_scale(
            magnitude_spi_ds[f'drought_magnitude_spi_filtered'],
            magnitude_ssi_ds[f'drought_magnitude_ssi_filtered'],
            magnitude_spi_time_scale,
            magnitude_ssi_time_scale
        )

        # Apply land mask
        magnitude_spi_data = magnitude_spi_data.where(land_mask == 1)
        magnitude_ssi_data = magnitude_ssi_data.where(land_mask == 1)

        # Calculate and save correlation for each lag
        for lag in tqdm(range(1, max_lag + 1), desc=f"Processing lags for SPI {magnitude_spi_time_scale} & SSI {magnitude_ssi_time_scale}", unit="lag"):
            corr_values = calculate_cross_correlation(magnitude_spi_data.values, magnitude_ssi_data.values, lag)
            corr_da = xr.DataArray(corr_values, coords={'lat': magnitude_spi_data.lat, 'lon': magnitude_spi_data.lon}, dims=['lat', 'lon'])
            corr_ds = xr.Dataset({'correlation': corr_da})
            output_file = os.path.join(correlation_folder, f"idn_cli_correlation_{time_scale_folder}_lag{lag:02d}.nc")
            corr_ds.to_netcdf(output_file)

        print(f"Time scale {magnitude_spi_time_scale} & {magnitude_ssi_time_scale} processing completed. Output saved.")

print("\nCross-correlation analysis completed for all time scale combinations.")


Processing SPI (Time Scale: 3) and SSI (Time Scale: 3)


Processing lags for SPI 3 & SSI 3: 100%|██████████████████████████████████████████████████████| 12/12 [20:01<00:00, 100.14s/lag]


Time scale 3 & 3 processing completed. Output saved.
Processing SPI (Time Scale: 3) and SSI (Time Scale: 6)


Processing lags for SPI 3 & SSI 6: 100%|███████████████████████████████████████████████████████| 12/12 [19:30<00:00, 97.51s/lag]


Time scale 3 & 6 processing completed. Output saved.
Processing SPI (Time Scale: 3) and SSI (Time Scale: 9)


...

Time scale 12 & 6 processing completed. Output saved.
Processing SPI (Time Scale: 12) and SSI (Time Scale: 9)


Processing lags for SPI 12 & SSI 9: 100%|██████████████████████████████████████████████████████| 12/12 [19:06<00:00, 95.58s/lag]


Time scale 12 & 9 processing completed. Output saved.
Processing SPI (Time Scale: 12) and SSI (Time Scale: 12)


Processing lags for SPI 12 & SSI 12: 100%|█████████████████████████████████████████████████████| 12/12 [19:11<00:00, 95.95s/lag]

Time scale 12 & 12 processing completed. Output saved.

Cross-correlation analysis completed for all time scale combinations.


## 4. Frequency Analysis

In the context of drought propagation analysis, frequency analysis plays a critical role in identifying the most prominent patterns of correlation between meteorological and hydrological drought indicators over time. By classifying cross-correlation values into distinct ranges (e.g., 0.0-0.1, 0.1-0.2, etc.) and analyzing these across different lag times, researchers can pinpoint the range that most frequently occurs. 

This approach helps in understanding the typical strength of correlation and the temporal shift (lag) between the onset of meteorological drought (as indicated by SPI) and its subsequent impact on hydrological conditions (as indicated by SSI). The most frequent range provides insights into the commonality of correlation strengths, while the corresponding lag sheds light on the typical delay between atmospheric changes and their effects on hydrological systems. 

**Let's convert the result into GeoTIFF files**

Getting statistical value within the netCDF files is faster, but it can be very slow if it required multiple arrays to be process. So far doing the similar analysis using GeoTIFF files as input is better.


In [15]:
import os
import rasterio
import xarray as xr
from rasterio.transform import Affine
from tqdm import tqdm

def convert_nc_to_geotiff(nc_file, geotiff_file):
    with xr.open_dataset(nc_file) as ds:
        # Assuming 'correlation' is the variable to be converted
        data = ds['correlation'].values
        lat = ds['lat'].values
        lon = ds['lon'].values

        # Calculate resolution
        res_lat = (lat[-1] - lat[0]) / (len(lat) - 1)
        res_lon = (lon[-1] - lon[0]) / (len(lon) - 1)

        # Check the order of the latitudes to determine if we need to flip the array
        if lat[0] > lat[-1]:  # latitudes are in descending order
            data = data[::-1, :]  # flip the data array vertically
            transform = Affine.translation(lon[0] - res_lon / 2, lat[-1] - res_lat / 2) * Affine.scale(res_lon, -res_lat)
        else:
            transform = Affine.translation(lon[0] - res_lon / 2, lat[0] - res_lat / 2) * Affine.scale(res_lon, res_lat)

        # Write to GeoTIFF
        with rasterio.open(
            geotiff_file,
            'w',
            driver='GTiff',
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=str(data.dtype),
            crs='+proj=latlong',
            transform=transform,
        ) as new_dataset:
            new_dataset.write(data, 1)

correlation_base_folder = './prop/04_correlation/'

# Iterate over each time scale folder
for time_scale_folder in os.listdir(correlation_base_folder):
    folder_path = os.path.join(correlation_base_folder, time_scale_folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {time_scale_folder}")
        for nc_file in tqdm(os.listdir(folder_path), desc="Converting files"):
            if nc_file.endswith('.nc'):
                nc_file_path = os.path.join(folder_path, nc_file)
                geotiff_file_path = nc_file_path.replace('.nc', '.tif')
                convert_nc_to_geotiff(nc_file_path, geotiff_file_path)

print("\nConversion to GeoTIFF completed for all files.")


Processing folder: spi03_ssi03


Converting files: 100%|█████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 16.04it/s]


Processing folder: spi03_ssi06


Converting files: 100%|█████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 17.87it/s]


Processing folder: spi03_ssi09


...

Processing folder: spi12_ssi09


Converting files: 100%|█████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 18.50it/s]


Processing folder: spi12_ssi12


Converting files: 100%|█████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 26.64it/s]


Conversion to GeoTIFF completed for all files.


**Calculate the statistics**

Calculate the maximum and the most frequent correlation, along with lag where the correlation is observe.

The script below will do:

* **Classify Correlation Values:** Classify the correlation values into bins (e.g., 0.0-0.1, 0.1-0.2, ... up to 1.0) for each lag.
* **Find the Most Frequent Range:** Determine the most frequently occurring range across all lags.
* **Identify the Most Frequent Lag:** Within the most frequent range, identify the most frequent lag.
* **Save the Results:** Save the most frequent range and corresponding lag as NetCDF files following CF conventions.


In [70]:
import os
import numpy as np
import rasterio
from tqdm import tqdm

# Define folders
correlation_base_folder = './prop/04_correlation/'
temp_output_folder = './prop/05_frequency/temp/'
max_frequency_folder = './prop/05_frequency/max/'
mode_frequency_folder = './prop/05_frequency/mode/'

# Ensure the output folder exists
os.makedirs(temp_output_folder, exist_ok=True)

# Time scale combinations
time_scale_combinations = [
    "spi03_ssi03", "spi06_ssi03", "spi09_ssi03", "spi12_ssi03",
    "spi03_ssi06", "spi06_ssi06", "spi09_ssi06", "spi12_ssi06",
    "spi03_ssi09", "spi06_ssi09", "spi09_ssi09", "spi12_ssi09",
    "spi03_ssi12", "spi06_ssi12", "spi09_ssi12", "spi12_ssi12"
]

def process_time_scale_folder(time_scale_folder):
    folder_path = os.path.join(correlation_base_folder, time_scale_folder)

    # Initialize arrays to store maximum and most frequent values
    max_corr_value = None
    max_corr_lag = None
    freq_corr_bins = None
    freq_corr_lag = None

    # Process each lag file
    for lag in range(1, 13):
        file_path = os.path.join(folder_path, f"idn_cli_correlation_{time_scale_folder}_lag{str(lag).zfill(2)}.tif")
        if os.path.exists(file_path):
            with rasterio.open(file_path) as src:
                corr_data = src.read(1)

                # Initialize arrays on the first iteration
                if max_corr_value is None:
                    max_corr_value = np.copy(corr_data)
                    max_corr_lag = np.full(corr_data.shape, 1, dtype=np.float32)  # Initialize with 1
                    freq_corr_bins = np.zeros((corr_data.shape[0], corr_data.shape[1], 10), dtype=np.int32)
                    freq_corr_lag = np.full(corr_data.shape, np.nan, dtype=np.float32)  # Initialize as float32 with NaN
                    profile = src.profile

                # Update maximum correlation and lag
                valid_mask = ~np.isnan(corr_data)
                new_max = (corr_data > max_corr_value) & valid_mask
                max_corr_value[new_max] = corr_data[new_max]
                max_corr_lag[new_max] = lag

                # Update frequency bins
                bin_indices = np.floor(corr_data * 10).astype(np.int32)
                bin_indices = np.clip(bin_indices, 0, 9)  # Ensure bin indices are within the range [0, 9]
                np.add.at(freq_corr_bins, (np.arange(corr_data.shape[0])[:, None], np.arange(corr_data.shape[1]), bin_indices), 1)

    # Calculate the most frequent correlation range and its lag
    freq_corr_value = np.nanmax(freq_corr_bins, axis=2) / 12.0
    freq_corr_lag = np.argmax(freq_corr_bins, axis=2) + 1

    # Save results in the temporary folder
    for name, data in [('max_corr_value', max_corr_value), ('max_corr_lag', max_corr_lag),
                       ('freq_corr_value', freq_corr_value), ('freq_corr_lag', freq_corr_lag)]:
        output_file = os.path.join(temp_output_folder, f'idn_cli_{name}_{time_scale_folder}.tif')
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(data, 1)

for time_scale_folder in tqdm(time_scale_combinations, desc="Processing time scale combinations"):
    process_time_scale_folder(time_scale_folder)

print("\nFrequency analysis completed for all time scale combinations.")


Processing time scale combinations:   0%|                                                                | 0/16 [00:00<?, ?it/s]/var/folders/9k/rftc85d17g90vcrwv_ctcd140000gp/T/ipykernel_78724/511869154.py:54: RuntimeWarning: invalid value encountered in cast
  bin_indices = np.floor(corr_data * 10).astype(np.int32)
Processing time scale combinations: 100%|███████████████████████████████████████████████████████| 16/16 [00:10<00:00,  1.49it/s]


Frequency analysis completed for all time scale combinations.


**Remove the NaN**

From above process, the output still have unwanted value, usually in non land areas. Let's remove it.


In [72]:
import rasterio
import os
import shutil

# Define folders
max_frequency_folder = './prop/05_frequency/max/'
mode_frequency_folder = './prop/05_frequency/mode'
temp_output_folder = './prop/05_frequency/temp/'

# Ensure the output folders exist
os.makedirs(max_frequency_folder, exist_ok=True)
os.makedirs(mode_frequency_folder, exist_ok=True)

# Define folders and combinations
correlation_base_folder = './prop/04_correlation/'
time_scale_combinations = [
    "spi03_ssi03", "spi06_ssi03", "spi09_ssi03", "spi12_ssi03",
    "spi03_ssi06", "spi06_ssi06", "spi09_ssi06", "spi12_ssi06",
    "spi03_ssi09", "spi06_ssi09", "spi09_ssi09", "spi12_ssi09",
    "spi03_ssi12", "spi06_ssi12", "spi09_ssi12", "spi12_ssi12"
]

# Choose a sample file from one of the time scale combinations to create the land mask
sample_time_scale = time_scale_combinations[0]  # Using the first time scale combination for the sample
sample_folder_path = os.path.join(correlation_base_folder, sample_time_scale)

# Find the first .tif file in the sample folder
for file in os.listdir(sample_folder_path):
    if file.endswith('.tif'):
        sample_file_path = os.path.join(sample_folder_path, file)
        break

# Load the sample file to create a land mask
with rasterio.open(sample_file_path) as mask_src:
    land_mask = ~np.isnan(mask_src.read(1))  # Non-land areas will be NaN

# Iterate through the files in the temp_output_folder
for filename in os.listdir(temp_output_folder):
    if filename.endswith('.tif'):
        file_path = os.path.join(temp_output_folder, filename)

        with rasterio.open(file_path) as src:
            data = src.read(1)
            profile = src.profile

            # Apply the land mask to the data
            data[~land_mask] = np.nan  # Set non-land areas to NaN

            # Determine the output folder
            if 'max_corr' in filename:
                output_folder = max_frequency_folder
            elif 'freq_corr' in filename:
                output_folder = mode_frequency_folder
            else:
                continue  # Skip files that don't match the criteria

            # Save the cleaned data
            output_file_path = os.path.join(output_folder, filename)
            with rasterio.open(output_file_path, 'w', **profile) as dst:
                dst.write(data, 1)

print("\nCleaned GeoTIFF outputs saved to respective folders.")

# Remove all files in the temp_output_folder
for filename in os.listdir(temp_output_folder):
    file_path = os.path.join(temp_output_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("\nTemporary files have been removed.")



Cleaned GeoTIFF outputs saved to respective folders.

Temporary files have been removed.


## 5. Visualisation

The visualization script is designed to illustrate the results of the cross-correlation analysis between meteorological (SPI) and hydrological (SSI) droughts. Key features of the script include:

* **Lag Map Creation:** This map displays the time lag (in months) between meteorological and hydrological droughts across the study area. It helps identify regions where hydrological responses to meteorological changes are immediate or delayed.

* **Strength Map Generation:** This map shows the strength of the correlation between meteorological and hydrological droughts. It highlights areas with a strong predictive relationship, indicating regions sensitive to meteorological changes.

* **Spatial Insight:** Both maps together provide a spatial understanding of drought dynamics, highlighting areas of resilience and vulnerability.

* **Practical Application:** The visualizations aid in drought management by pinpointing critical areas for focused attention and intervention.


In [114]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import rasterio
import os
import numpy as np
import matplotlib.colors as mcolors

def plot_maps(folder, combinations, file_prefix, main_title, output_folder):
    # Normalization for the colorbars
    value_norm = mcolors.BoundaryNorm(boundaries=np.linspace(0, 1.0, 11), ncolors=10)
    lag_norm = mcolors.BoundaryNorm(boundaries=np.arange(1, 13), ncolors=12)

    # Define color maps for discrete intervals
    value_cmap = plt.get_cmap('inferno', 10)
    lag_cmap = plt.get_cmap('coolwarm', 12)

    for i in range(0, len(combinations), 4):
        subset_combinations = combinations[i:i+4]
        # Adjust figsize to make the plot more compact
        fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

        for j, time_scale_folder in enumerate(subset_combinations):
            # Load data
            value_file = os.path.join(folder, f'idn_cli_{file_prefix}_value_{time_scale_folder}.tif')
            lag_file = os.path.join(folder, f'idn_cli_{file_prefix}_lag_{time_scale_folder}.tif')

            with rasterio.open(value_file) as src:
                value_data = src.read(1)
                extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

            with rasterio.open(lag_file) as src:
                lag_data = src.read(1)

            # Plot max_corr_value
            ax_value = axs[j, 0]
            ax_value.set_extent(extent, crs=ccrs.PlateCarree())
            ax_value.coastlines()
            im_value = ax_value.imshow(value_data, extent=extent, transform=ccrs.PlateCarree(), cmap=value_cmap, norm=value_norm, origin='upper')
            ax_value.set_title(f'{time_scale_folder} - Strength', fontsize=10)
            
            # Plot max_corr_lag
            ax_lag = axs[j, 1]
            ax_lag.set_extent(extent, crs=ccrs.PlateCarree())
            ax_lag.coastlines()
            im_lag = ax_lag.imshow(lag_data, extent=extent, transform=ccrs.PlateCarree(), cmap=lag_cmap, norm=lag_norm, origin='upper')
            ax_lag.set_title(f'{time_scale_folder} - Lag', fontsize=10)

        plt.suptitle(main_title, fontsize=12, y=0.98)

        # Configure and place the colorbar
        cbar_ax_value = fig.add_axes([0.125, 0.07, 0.35, 0.01])
        cbar_ax_lag = fig.add_axes([0.575, 0.07, 0.35, 0.01])
        cbar_value = fig.colorbar(im_value, cax=cbar_ax_value, orientation='horizontal', spacing='proportional')
        cbar_lag = fig.colorbar(im_lag, cax=cbar_ax_lag, orientation='horizontal', spacing='proportional')

        cbar_value.set_label('Correlation Strength', fontsize=10)
        cbar_lag.set_label('Lag (months)', fontsize=10)

        # Adjust layout to make room for colorbar and reduce spaces between rows
        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, hspace=0.05, wspace=0.1)
        # plt.tight_layout(rect=[0, 0.03, 1, 0.95], pad=1.8)

        print('Saving the plot...')
        plt.savefig(os.path.join(output_folder, f'idn_cli_{file_prefix}_combination_{i//4 + 1}.png'), dpi=300)
        plt.close()

# Define folders and combinations
max_frequency_folder = './prop/05_frequency/max/'
mode_frequency_folder = './prop/05_frequency/mode/'
time_scale_combinations = [
    "spi03_ssi03", "spi06_ssi03", "spi09_ssi03", "spi12_ssi03",
    "spi03_ssi06", "spi06_ssi06", "spi09_ssi06", "spi12_ssi06",
    "spi03_ssi09", "spi06_ssi09", "spi09_ssi09", "spi12_ssi09",
    "spi03_ssi12", "spi06_ssi12", "spi09_ssi12", "spi12_ssi12"
]
output_folder = './prop/images/'
os.makedirs(output_folder, exist_ok=True)

# Plotting for max_corr
print(f'Creating plot on the Maximum Correlation for {time_scale_combinations}')
plot_maps(max_frequency_folder, time_scale_combinations, 'max_corr', 
          'The Strength and Lag of the Maximum Correlation between Meteorological and Hydrological Droughts', output_folder)

# Plotting for freq_corr
print(f'Creating plot on the Most Frequent Correlation for {time_scale_combinations}')
plot_maps(mode_frequency_folder, time_scale_combinations, 'freq_corr', 
          'The Strength and Lag of the Most Frequent Correlation between Meteorological and Hydrological Droughts', output_folder)

print('Completed!')


Creating plot on the Maximum Correlation for ['spi03_ssi03', 'spi06_ssi03', 'spi09_ssi03', 'spi12_ssi03', 'spi03_ssi06', 'spi06_ssi06', 'spi09_ssi06', 'spi12_ssi06', 'spi03_ssi09', 'spi06_ssi09', 'spi09_ssi09', 'spi12_ssi09', 'spi03_ssi12', 'spi06_ssi12', 'spi09_ssi12', 'spi12_ssi12']
Saving the plot...
Saving the plot...
Saving the plot...
Saving the plot...
Creating plot on the Most Frequent Correlation for ['spi03_ssi03', 'spi06_ssi03', 'spi09_ssi03', 'spi12_ssi03', 'spi03_ssi06', 'spi06_ssi06', 'spi09_ssi06', 'spi12_ssi06', 'spi03_ssi09', 'spi06_ssi09', 'spi09_ssi09', 'spi12_ssi09', 'spi03_ssi12', 'spi06_ssi12', 'spi09_ssi12', 'spi12_ssi12']
Saving the plot...
Saving the plot...
Saving the plot...
Saving the plot...
Completed!


End of notebook